In [1]:
##
# Lets create the file
##

import turicreate as tc
import csv

with open('timeFramedTrainingData.csv', 'rb') as trainingData:
    sf = tc.SFrame.read_csv('timeFramedTrainingData.csv', delimiter=',', header=True, verbose=False)
        
sf.save('timeFramedTrainingData.sframe')

with open('timeFramedValidationData.csv', 'rb') as validationData :
    vsf = tc.SFrame.read_csv('timeFramedValidationData.csv', delimiter=',', header=True, verbose=False)
    
vsf.save('timeFramedValidationData.sframe')


In [ ]:
##
# Let manipulate said file - Oh yea
##

import turicreate as tc

# Load sessions from preprocessed data
data = tc.SFrame('timeFramedValidationData.sframe')
validationSet = tc.SFrame('timeFramedValidationData.sframe')

# Train/test split by recording sessions
train, test = tc.activity_classifier.util.random_split_by_session(data, session_id='id', fraction=0.8)

# Create an activity classifier
model = tc.activity_classifier.create(train, session_id='id', prediction_window=4, target='type', validation_set=validationSet, max_iterations=100)

# Save the model for later use in Turi Create
model.save('timeFramedActivityClassifier.model')

# Export for use in Core ML
model.export_coreml('timeFramedActivityClassifier.mlmodel')

# Evaluate the model and save the results into a dictionary (might not work as data is not split evenly by classes)
metrics = model.evaluate(validationSet)
model.summary('stdout')
print(metrics['accuracy'])

Pre-processing 676 samples...

Using sequences of size 80 for model creation.

Processed a total of 169 sessions.

Iteration: 0001
	Train loss    : 0.320564372 	Train accuracy: 0.070601107
	Valid loss    : 0.120137010 	Valid accuracy: 0.509428866
Iteration: 0002
	Train loss    : 0.158517758 	Train accuracy: 0.294557132
	Valid loss    : 0.106542675 	Valid accuracy: 0.533013535
Iteration: 0003
	Train loss    : 0.117140235 	Train accuracy: 0.401616188
	Valid loss    : 0.094570036 	Valid accuracy: 0.566032075
Iteration: 0004
	Train loss    : 0.096902824 	Train accuracy: 0.429972380
	Valid loss    : 0.083249224 	Valid accuracy: 0.646219937
Iteration: 0005
	Train loss    : 0.079662111 	Train accuracy: 0.490735610
	Valid loss    : 0.073366289 	Valid accuracy: 0.688672345
Iteration: 0006
	Train loss    : 0.071667453 	Train accuracy: 0.553813631
	Valid loss    : 0.064020808 	Valid accuracy: 0.721690884
Iteration: 0007
	Train loss    : 0.061460361 	Train accuracy: 0.596058538
	Valid loss    : 0.056687827 	Valid accuracy: 0.721690884
Iteration: 0008
	Train loss    : 0.068417180 	Train accuracy: 0.548605348
	V

	Valid loss    : 0.029030698 	Valid accuracy: 0.853765038
Iteration: 0064
	Train loss    : 0.013213277 	Train accuracy: 0.932281872
	Valid loss    : 0.029612340 	Valid accuracy: 0.853765038
Iteration: 0065
	Train loss    : 0.016075500 	Train accuracy: 0.880199095
	Valid loss    : 0.030573967 	Valid accuracy: 0.844331166
Iteration: 0066
	Train loss    : 0.015119333 	Train accuracy: 0.892930448
	Valid loss    : 0.029203430 	Valid accuracy: 0.853765029
Iteration: 0067
	Train loss    : 0.015278291 	Train accuracy: 0.901032219
	Valid loss    : 0.028486232 	Valid accuracy: 0.858481965
Iteration: 0068
	Train loss    : 0.011845259 	Train accuracy: 0.927073608
	Valid loss    : 0.028360514 	Valid accuracy: 0.853765029
Iteration: 0069
	Train loss    : 0.012166251 	Train accuracy: 0.911448767
	Valid loss    : 0.028276642 	Valid accuracy: 0.858481960
Iteration: 0070
	Train loss    : 0.013978840 	Train accuracy: 0.916657041
	Valid loss    : 0.028234767 	Valid accuracy: 0.858481960
Iteration: 0071
	T

In [12]:
###
# Model Test
###

dat = validationSet[validationSet['id'] == '17F6FE84-F150-45CF-834C-79E10BC3A8CB']
pre = model.predict(dat, output_frequency='per_window')
print(metrics['accuracy'], pre)
model.evaluate(test)

(0.9150943396226415, Columns:
	prediction_id	int
	id	str
	class	str

Rows: 1

Data:
+---------------+-------------------------------+-----------------+
| prediction_id |               id              |      class      |
+---------------+-------------------------------+-----------------+
|       0       | 17F6FE84-F150-45CF-834C-79... | rotateRightBack |
+---------------+-------------------------------+-----------------+
[1 rows x 3 columns]
)


ToolkitError: Size of prediction probability vector(15) != number of classes(14).


type,xaccl,yaccl,zaccl,xmagnet,ymagnet,zmagnet,xgyro,ygyro,zgyro,id
